In [1]:
import numpy as np
import keras.utils.np_utils as kutils
from sklearn.model_selection import train_test_split
import keras.callbacks as callbacks
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.utils import plot_model
from keras.optimizers import SGD

C:\Users\y884wang\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
airplane = np.load('airplane.npy')
axe = np.load('axe.npy')
bed = np.load('bed.npy')
bicycle = np.load('bicycle.npy')
butterfly = np.load('butterfly.npy')
envelope = np.load('envelope.npy')
knife = np.load('knife.npy')
square = np.load('square.npy')
star = np.load('star.npy')
donut = np.load('donut.npy')

In [3]:
airplane = np.c_[airplane, np.zeros(airplane.shape[0])]
axe = np.c_[axe, np.ones(axe.shape[0])]
bed = np.c_[bed, np.ones(bed.shape[0])*2]
bicycle  = np.c_[bicycle , np.ones(bicycle .shape[0])*3]
butterfly = np.c_[butterfly, np.ones(butterfly.shape[0])*4]
envelope  = np.c_[envelope, np.ones(envelope.shape[0])*5]
knife = np.c_[knife, np.ones(knife.shape[0])*6]
square = np.c_[square, np.ones(square.shape[0])*7]
star = np.c_[star, np.ones(star.shape[0])*8]
donut = np.c_[donut, np.ones(donut.shape[0])*9]

In [4]:
Data = np.concatenate((airplane, axe,bed,bicycle ,butterfly,envelope,knife,square,star,donut),axis=0)
del airplane,axe,bed,bicycle ,butterfly,envelope,knife,square,star,donut
X = Data[:,0:784]
Y = Data[:,784]
del Data

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [6]:
del X, Y

In [7]:
X_train = X_train.reshape(-1, 28,28, 1)

In [8]:
X_train.shape

(1076133, 28, 28, 1)

In [9]:
X_test = X_test.reshape(-1,28,28,1)

In [10]:
X_test.shape

(269034, 28, 28, 1)

In [11]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.
X_test = X_test / 255.

In [12]:
y_train = kutils.to_categorical(Y_train)
y_test = kutils.to_categorical(Y_test)

In [13]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=13)

In [14]:
X_valid.shape

(107614, 28, 28, 1)

In [15]:
y_train.shape

(968519, 10)

In [16]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [17]:
batch_size = 128
epochs = 8
num_classes = 10

In [18]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [19]:
callbacks = [ModelCheckpoint('weights',
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                            mode='auto')
            ]
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(), metrics=["acc"])
model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs,
                   callbacks=callbacks,
                   validation_data=(X_valid, y_valid))

Train on 968519 samples, validate on 107614 samples
Epoch 1/20
968519/968519 [==============================] - 1139s 1ms/step - loss: 0.1624 - acc: 0.9525 - val_loss: 0.1156 - val_acc: 0.9658

Epoch 00001: val_acc improved from -inf to 0.96576, saving model to weights
Epoch 2/20
968519/968519 [==============================] - 1166s 1ms/step - loss: 0.1085 - acc: 0.9675 - val_loss: 0.1044 - val_acc: 0.9689

Epoch 00002: val_acc improved from 0.96576 to 0.96894, saving model to weights
Epoch 3/20
968519/968519 [==============================] - 1173s 1ms/step - loss: 0.0946 - acc: 0.9713 - val_loss: 0.1004 - val_acc: 0.9698

Epoch 00003: val_acc improved from 0.96894 to 0.96980, saving model to weights
Epoch 4/20
968519/968519 [==============================] - 1185s 1ms/step - loss: 0.0850 - acc: 0.9740 - val_loss: 0.1042 - val_acc: 0.9694

Epoch 00004: val_acc did not improve
Epoch 5/20
968519/968519 [==============================] - 1187s 1ms/step - loss: 0.0770 - acc: 0.9762 - val

KeyboardInterrupt: 

In [20]:
test_eval = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

Test loss: 0.13996616562194816
Test accuracy: 0.9680783841447549
